# Hybrid Streaming Optimizer API Demo

This notebook demonstrates how to use `method='hybrid_streaming'` with both
`curve_fit()` and `curve_fit_large()` functions.

The hybrid streaming optimizer provides:
- Parameter normalization for better gradient signals
- Adam warmup for robust initial convergence
- Streaming Gauss-Newton for exact covariance computation
- Automatic memory management for large datasets

In [ ]:
# Configure matplotlib for inline plotting in VS Code/Jupyter
# MUST come before importing matplotlib
%matplotlib inline

In [ ]:
import numpy as np
import jax.numpy as jnp
from nlsq import curve_fit, curve_fit_large

In [ ]:
def exponential_decay(x, a, b, c):
    """Three-parameter exponential decay model."""
    return a * jnp.exp(-b * x) + c

In [ ]:
# Generate synthetic data
np.random.seed(42)
x = np.linspace(0, 10, 2000)
true_params = np.array([5.0, 0.5, 1.0])
y_true = exponential_decay(x, *true_params)
y = y_true + np.random.normal(0, 0.1, len(x))

print(f"Dataset: {len(x)} samples")
print(f"True parameters: a={true_params[0]}, b={true_params[1]}, c={true_params[2]}")

## Example 1: Basic usage with curve_fit()

In [ ]:
result = curve_fit(
    exponential_decay,
    x,
    y,
    p0=np.array([4.0, 0.4, 0.8]),
    method='hybrid_streaming',
    verbose=1,
)

# Unpack result
popt, pcov = result

print(f"\nFitted parameters: {popt}")
print(f"True parameters:   {true_params}")
print(f"Parameter errors:  {np.abs(popt - true_params)}")
print(f"\nCovariance matrix diagonal: {np.diag(pcov)}")
print(f"Parameter std errors: {np.sqrt(np.diag(pcov))}")

## Example 2: With parameter bounds

In [ ]:
result = curve_fit(
    exponential_decay,
    x,
    y,
    p0=np.array([4.0, 0.4, 0.8]),
    bounds=([0, 0, 0], [10, 2, 5]),
    method='hybrid_streaming',
    verbose=0,  # Silent mode
)

popt, pcov = result
print(f"Fitted parameters (bounded): {popt}")
print(f"Within bounds: {np.all(popt >= [0, 0, 0]) and np.all(popt <= [10, 2, 5])}")

## Example 3: Large dataset with curve_fit_large()

In [ ]:
# Generate larger dataset
x_large = np.linspace(0, 10, 10000)
y_large = exponential_decay(x_large, *true_params) + np.random.normal(0, 0.1, len(x_large))

popt, pcov = curve_fit_large(
    exponential_decay,
    x_large,
    y_large,
    p0=np.array([4.0, 0.4, 0.8]),
    method='hybrid_streaming',
    verbose=1,
)

print(f"\nFitted parameters (large dataset): {popt}")
print(f"True parameters:                   {true_params}")
print(f"Parameter errors:                  {np.abs(popt - true_params)}")

## Example 4: Custom configuration via kwargs

In [ ]:
result = curve_fit(
    exponential_decay,
    x,
    y,
    p0=np.array([4.0, 0.4, 0.8]),
    method='hybrid_streaming',
    verbose=0,
    # HybridStreamingConfig overrides:
    warmup_iterations=300,
    normalization_strategy='bounds',
    phase2_max_iterations=100,
)

popt, pcov = result
print(f"Fitted parameters (custom config): {popt}")
print(f"Result attributes available: {list(result.keys())[:10]}")

## Example 5: Accessing full result details

In [ ]:
result = curve_fit(
    exponential_decay,
    x,
    y,
    p0=np.array([4.0, 0.4, 0.8]),
    method='hybrid_streaming',
    verbose=0,
)

print(f"Success: {result['success']}")
print(f"Message: {result['message']}")
print(f"Final cost: {result.get('cost', 'N/A')}")

if 'streaming_diagnostics' in result:
    diag = result['streaming_diagnostics']
    print(f"\nStreaming diagnostics available:")
    print(f"  Keys: {list(diag.keys())}")

## Summary

The hybrid streaming optimizer combines:
- **Parameter normalization** for better gradient signals when parameters have different scales
- **Adam warmup** for robust initial convergence from poor initial guesses
- **Streaming Gauss-Newton** for fast second-order convergence near the optimum
- **Exact covariance computation** for reliable uncertainty estimates

Use `method='hybrid_streaming'` when:
- Parameters span many orders of magnitude
- Large datasets (100K+ points) with memory constraints
- Need production-quality uncertainty estimates
- Standard optimizers converge slowly